In [1]:
!pip install datasets #provivides efficient data pre-processing and one-line dataloaders for public datasets

#Specific sentiment analysis model: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/torchtext_0_10_imdb_nbow.ipynb
#General GitHub: https://github.com/bentrevett/pytorch-sentiment-analysis

#Deliverables: Take 6-10 random rows and run it though the model and show predictions and actual
#Understand model?
#All PyTorch deep learning is good for learning
#Optomize

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd

import torch #Entire library
import torch.nn as nn                           #Neural network modules (linear, convolutional, etc)
import torch.optim as optim                     #Optimization algorithms
import torch.nn.functional as F                 #Functions with no parameters (activation functions like relu)
#from torch.utils.data import DataLoader,Dataset #Easier dataset managment (mini batches etc)
import torchvision.datasets as datasets         #Available datasets (WONT USE)
import torchvision.transforms as transforms     #Transformations on dataset
#import torchtext
import datasets
import functools

#from google.colab import drive
#drive.mount('/content/gdrive')
#from google.colab import files

from sklearn.feature_extraction.text import CountVectorizer #Break down text
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence #optomize computations and https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch
from tqdm import tqdm, tqdm_notebook            #Generation of progress bars

/shared-libs/python3.9/py/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /shared-libs/python3.9/py/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator12recordStreamERKNS_7DataPtrENS0_10CUDAStreamE
  warn(f"Failed to load image Python extension: {e}")
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install torchtext

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
#df = pd.read_csv('gdrive/My Drive/ACM Research DisinfoDeception/labeled_data.csv')
df = pd.read_csv("/work/labeled_data.csv")
df0 = df.replace({'class': 1}, 0)
df2 = df.replace({'class': 1}, 2)
#ds0 = df0.to_xarray()

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
#from google.colab import drive
#from google.colab import files
#drive.mount('/content/drive/')

In [7]:
#Removes unnecesscary leading characters
def remove_tags(text):
  text = text[text.find(":") + 1:]
  return text


Class 2: none

Class 1: offensive

Class 0: hate speech

In [8]:
from datasets import Dataset, DatasetDict
df_train_data = df0.sample(frac = 0.8)
df_test_data = df0.drop(df_train_data.index)
train_data = Dataset.from_pandas(df_train_data)
test_data = Dataset.from_pandas(df_test_data)
#train_data, test_data= datasets.load_dataset("hate_speech_offensive", split=['train[:80%]', 'train[:20%]'])

In [9]:
train_data, test_data

(Dataset({
     features: ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet', '__index_level_0__'],
     num_rows: 19826
 }),
 Dataset({
     features: ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet', '__index_level_0__'],
     num_rows: 4957
 }))

In [10]:
train_data = train_data.map(remove_columns=['count', 'hate_speech', 'offensive_language', 'neither', 'Unnamed: 0', '__index_level_0__'])
test_data = test_data.map(remove_columns=['count', 'hate_speech', 'offensive_language', 'neither', 'Unnamed: 0', '__index_level_0__'])
train_data, test_data

(Dataset({
     features: ['class', 'tweet'],
     num_rows: 19826
 }),
 Dataset({
     features: ['class', 'tweet'],
     num_rows: 4957
 }))

In [11]:
import torchtext
from torchtext import data
from torchtext import datasets

In [12]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

In [13]:
#Break up words
def tokenize_data(example, tokenizer):
    tokens = {'tokens': tokenizer(example['tweet'])}
    return tokens

In [14]:
#takes each tweet in the respective datasets and tokenizes it, creates new column in dataset
train_data = train_data.map(tokenize_data, fn_kwargs={'tokenizer': tokenizer})
test_data = test_data.map(tokenize_data, fn_kwargs={'tokenizer': tokenizer})

In [15]:
train_data, test_data

(Dataset({
     features: ['class', 'tweet', 'tokens'],
     num_rows: 19826
 }),
 Dataset({
     features: ['class', 'tweet', 'tokens'],
     num_rows: 4957
 }))

In [16]:
train_valid_data = train_data.train_test_split(test_size=0.25)
train_data = train_valid_data['train']
valid_data = train_valid_data['test']

In [17]:
len(train_data), len(valid_data), len(test_data)

(14869, 4957, 4957)

In [18]:
min_freq = 3 #look into modifying this
special_tokens = ['<unk>', '<pad>'] #unk is used for unknown word and pad is for padding

vocab = torchtext.vocab.build_vocab_from_iterator(train_data['tokens'],
                                                  min_freq=min_freq,
                                                  specials=special_tokens)

In [19]:
vocab.get_itos()[:10] #List mapping indices to tokens

['<unk>', '<pad>', '.', "'", 'i', 'a', 'bitch', 'rt', 'the', 'you']

In [20]:
unk_index = vocab['<unk>']

unk_index

0

In [21]:
pad_index = vocab['<pad>']

pad_index

1

In [22]:
vocab.set_default_index(unk_index) #what does this do lol

In [23]:
def numericalize_data(example, vocab):
    ids = {'ids': [vocab[token] for token in example['tokens']]}
    return ids

In [24]:
train_data = train_data.map(numericalize_data, fn_kwargs={'vocab': vocab})
valid_data = valid_data.map(numericalize_data, fn_kwargs={'vocab': vocab})
test_data = test_data.map(numericalize_data, fn_kwargs={'vocab': vocab})

In [25]:
train_data.set_format(type='torch', columns=['ids', 'class']) #torch format (rather than numpy or something else)
valid_data.set_format(type='torch', columns=['ids', 'class'])
test_data.set_format(type='torch', columns=['ids', 'class'])
train_data, valid_data, test_data

(Dataset({
     features: ['class', 'tweet', 'tokens', 'ids'],
     num_rows: 14869
 }),
 Dataset({
     features: ['class', 'tweet', 'tokens', 'ids'],
     num_rows: 4957
 }),
 Dataset({
     features: ['class', 'tweet', 'tokens', 'ids'],
     num_rows: 4957
 }))

In [26]:
class NBoW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_index):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index) #Embedding layer
        self.fc = nn.Linear(embedding_dim, output_dim) #Linear layer

    def forward(self, text):
        # text = [batch size, seq len]
        embedded = self.embedding(text)
        # embedded = [batch size, seq len, embedding dim]
        pooled = embedded.mean(dim=1)
        # pooled = [batch size, embedding dim]
        prediction = self.fc(pooled)
        # prediction = [batch size, output dim]
        return torch.sigmoid(prediction)

In [27]:
vocab_size = len(vocab)
embedding_dim = 256
output_dim = 1

model = NBoW(vocab_size, embedding_dim, output_dim, pad_index)

In [28]:
vectors = torchtext.vocab.GloVe() #Wow, WHAT IS THIS

.vector_cache/glove.840B.300d.zip: 2.18GB [06:50, 5.31MB/s]                            


OSError: [Errno 122] Disk quota exceeded

In [0]:
optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def collate(batch, pad_index):
    batch_ids = [i['ids'] for i in batch]
    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    batch_labels = [i['class'] for i in batch]
    batch_labels = torch.stack(batch_labels)
    batch = {'ids': batch_ids,
             'class': batch_labels}
    return batch

In [0]:
batch_size = 512

collate = functools.partial(collate, pad_index=pad_index)

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, collate_fn=collate)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, collate_fn=collate)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, collate_fn=collate)
train_dataloader

In [0]:
def get_accuracy(predictions, labels):
    batch_size = predictions.shape[0]
    predicted_classes = predictions.argmax(1, keepdim=True)
    correct_predictions = predicted_classes.eq(labels.view_as(predicted_classes)).sum()
    accuracy = correct_predictions.float() / batch_size
    return accuracy

In [0]:
def train(dataloader, model, criterion, optimizer, device):

    model.train()
    epoch_loss = 0
    epoch_accuracy = 0

    for batch in dataloader:
        tokens = batch['ids'].to(device)
        labels = batch['class'].to(device)
        predictions = model(tokens)
        #predictions = predictions.unsqueeze(1)
        #predictions = predictions.float()
        #labels = labels.unsqueeze(1)
        #labels = labels.float()
        loss = criterion(predictions.squeeze(), labels.float())
        accuracy = get_accuracy(predictions, labels) #function defined above
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()

    return epoch_loss / len(dataloader), epoch_accuracy / len(dataloader)

In [0]:
def evaluate(dataloader, model, criterion, device):
    
    model.eval()
    epoch_loss = 0
    epoch_accuracy = 0

    with torch.no_grad():
        for batch in dataloader:
            tokens = batch['ids'].to(device)
            labels = batch['class'].to(device)
            predictions = model(tokens)
            loss = criterion(predictions.squeeze(), labels.float())
            accuracy = get_accuracy(predictions, labels)
            epoch_loss += loss.item()
            epoch_accuracy += accuracy.item()

    return epoch_loss / len(dataloader), epoch_accuracy / len(dataloader)

In [0]:
n_epochs = 10 #output sentence, predicted, actual

for epoch in range(n_epochs):

    train_loss, train_acc = train(train_dataloader, model, criterion, optimizer, device)
    valid_loss, valid_acc = evaluate(valid_dataloader, model, criterion, device)

    print(f'epoch: {epoch+1}')
    print(f'train_loss: {train_loss:.3f}, train_acc: {train_acc:.3f}')
    print(f'valid_loss: {valid_loss:.3f}, valid_acc: {valid_acc:.3f}')

In [0]:
torch.save(model.state_dict(), '/content/drive/MyDrive/ACMResearch')

https://youtu.be/4jROlXH9Nvc

try RNN (recurrent neural network) Text classification (51:00) with embeddings? 

*   (difference is that output of same layers fed back into inputs)
*   Uses hidden layers (At each timestep, the rnn unit uses the previous hidden state and the input at the timestep to predict a new hiddent state and an output) h_t = f(h_(t-1), x_t; theta)

Problems: vanishing gradient and exploding gradient
 

*   vanishing gradient rnns can be long with timestepe and if gradients coming from early timesteps have to go through many matrix multiplications and if they have values < 1, they shrink exponentially. Meaning that your network will forget longer-term dependencies.
*   Exploding means that the gradients coming frmo deeper layers have values larger than 1, they will explode (gradient clipping helps remedy this)

RNN architectures: Simple RNN, LSTM (long short-term memory) gates and stuff, GRU (Gated recurrent unit)

Types of RNNs: 
* 1:1 image classification
* 1:N image captioning
* M:1 Sentiment classification (use this)
* M:N machine translation, text generation, video classification on a frame level

Embeddings: distributed representation of text.

*   Before text represented categorically (bag of words, one hot encoding). Assume nearby words share semantic emaning by association.
*   Idea: train a nn to predict a target word given a context word.
*   GloVe: commonly used pretrained word embeddings. It maps discrete categorical values to continuous space.
*   Download py magnitude for efficient embedding vector loading (49:30). Can be used to find similarities between words

Note: May want to disable gradient tracking when model is trained and just want to apply it to some input data

Possilbe with:
*   torch.nograd(): custom function
*   .detach() built in function



Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:

**The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.

**The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.


Stopped at 1:09:40

-------------------------------------------------------------------------
video notes(start until 45:00)
- build compotation graph (matrix mulitplication) in model then do gradient descent (set learning rate to find position in graph to minimize the loss with each training), define loss, feed in prediction
- backward: gradient of loss vs graph
  - x:input, w:weights
  - computes gradient on fly, stores gradient in the vars
- optimizer: knows which parameters to update for each(?) gradient which multiplies learning rate
    - (so is each gradient basically the change in value of the weights??)

- dataset class
  - read in data, tokenize data, vectorize data
  - returns input and target (what class it's supposed to be in (ex. hate) ??)

- model class (MODAL == COMPUTATIONAL GRAPH)
  - self.f1 = nn.Linear(128, 32) 
  - self.f1 = nn.Linear(16, 1) 
      - vars, not numbers
      - nn.Linear(16, 1): 1 means identifies as 1 or 0
        - **OH SO STARTS AT 128 (DOTS == neurons), THEN REDUCES TO 32 DOTS, THEN REDUCES TO 16 DOTS, THEN REDUCES TO 1 DOT THAT SIGNIFIES T/F WHERE 0 OR 1**
      - f1, f2, f3 are each of the layers/objects
  - he doing binary classifier of 0 and 1
  - forward
    - builds dynamic computational graph and calls it by sequence
    - feed input thru each layer
    - relu??: rectifier, takes max btn 0 and value, gets rid of negative, non-linear output
    - sigmoid: output btn 0 aand 1

- training
  - epoch == rounds of training (?)
  - loader: dataset
  - split dataset into batches, 
    - (batch size vs input size ??)
  - tensor?
  - 1) optimizer.zero_grad() : zeros out all the graidents bc dont want prev runs to determine current run
  - 2) forward pass: just running input through model
  - 3) loss: get how wrong prediction is
  - 4) GRADIENT DESCENT == BACKWARD PASS
    - (so determines how the gradients shld be set next based on the loss??)
  - 5)optimizer step
    - updates vars?

NLP
- tokenize dataset, remove useless/low-info words that would create bad bias, lemmatize (take tenses of words and simplifiy them to get them to the basic meaning to get more overlap on traiing set), get rid of non-freq words (masking words, not just get rid of it)
- vectorize tokenized dataset
  - get vocab (unique words), 2d matrix that shows where the word is (index) vs vocab, sums to get vector with occurance of each word, dict
    - bag of words cant tell synynyms, discrete, need continuous for synynyms, what CountVectorizer does

- **ONLY 1 CLASS SO USE BINARY-CROSS ENTROPHY, USE CROSS ENTROPGY FOR MULIT-CLASS**
- optim.Adam: dynamically changes leanring rate so not overcrossing(?)

- **basically u specificy # of epochs/training rounds, then for each round u input the data into the model, get the loss number which tells u how far off the model was from the correct prediciton, then takes a step backward which determines the numbers to set the gradients to in order to get a lower loss number in the next round, then u tell the optimizer to actually update the parameters with the gradients to improve the accuracy**
  - zero out the gradients, why: to make sure past gradients dont influence future gradients(?)




?
- gradient descent def
- optimize.step def
- did yall try to implement multiple layers
- what problems were encountered
- sequence: time (ex. some text goes after other text to give it meaning in sequence) (nn have trouble with this, rnn dont)
- tf is a hidden step?
- this is much pain :|





In [ ]:
torch.save(model.state_dict(), 'tut5-model.pt')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e803e9f0-f341-4e63-9e5c-ecfa91ec1687' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>